In [1]:
import requests
import json


In [2]:
response_API = requests.get('https://developer.nps.gov/api/v1/roadevents?type=workzone&api_key=pAVN008JKe6w3BaWr6FmH8jPFbzkAbKkv6ZpditQ')
#print(response_API.status_code)


In [3]:
data = response_API.text
df = json.loads(data)


In [4]:
df["road_event_feed_info"]

{'publisher': 'National Park Service',
 'version': '4.1',
 'update_date': '2023-04-28T06:37:22.497618Z',
 'update_frequency': 60,
 'contact_name': 'National Park Service',
 'contact_email': 'asknps@nps.gov',
 'data_sources': [{'data_source_id': 'de2fee5c-d31a-4bee-2d7f-08d4c7ad36e2',
   'organization_name': 'Olympic National Park',
   'update_date': '2023-07-16T09:30:15.13926Z',
   'contact_name': 'Olympic National Park',
   'contact_email': 'OLYM_Superintendent@nps.gov'}],
 'license': 'https://creativecommons.org/publicdomain/zero/1.0/',
 'id': '6ba84a20-3be7-4df0-81eb-711eccf84f18'}

In [5]:
df["features"]

[{'type': 'Feature',
  'id': '89715b7a-b3b9-4b71-bcec-f17a2a02bc61',
  'geometry': {'type': 'LineString',
   'coordinates': [[-124.03271608374007, 47.81796544329074],
    [-124.03239989301939, 47.818013019875195],
    [-124.03210715599019, 47.818074343910695],
    [-124.03187838697151, 47.81813900493916],
    [-124.03151450895389, 47.81829119198681],
    [-124.03073946592512, 47.818650592089845],
    [-124.03043554090125, 47.81874050212793],
    [-124.0297838958402, 47.818893076207665],
    [-124.02895915275931, 47.81907142730791],
    [-124.02863142173041, 47.81915534234839],
    [-124.02535926142856, 47.81994041774979],
    [-124.02470208236483, 47.82008574882985],
    [-124.02420137831344, 47.82018469289028],
    [-124.02354120325005, 47.82033289897082],
    [-124.02300830718887, 47.820411926033955],
    [-124.02220595106766, 47.82041290312354],
    [-124.02179091600858, 47.82042790617054],
    [-124.02012771676438, 47.82045851135762],
    [-124.01823608947205, 47.820433767567664],
